In [6]:
import pandas as pd

data = pd.read_csv('bakery_train_dataset.csv')

In [7]:
# Prepare a plain-text file for GPT-2 fine-tuning
with open('bakery_for_finetuning.txt', 'w', encoding='utf-8') as f:
    for _, row in data.iterrows():
        # Assuming the recipe text is in a column named "directions"
        if pd.notna(row['directions']):  # Skip NaN values
            f.write(row['directions'] + '\n<|endoftext|>\n')

print("Text file prepared: bakery_for_finetuning.txt")

Text file prepared: bakery_for_finetuning.txt


In [8]:
import accelerate
import huggingface_hub

print("Accelerate version:", accelerate.__version__)
print("Huggingface Hub version:", huggingface_hub.__version__)

Accelerate version: 1.1.1
Huggingface Hub version: 0.26.2


In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load GPT-2 tokenizer and model
model_name = './gpt2-finetuned-recipes-main'  # Use the small GPT-2 model to fit your hardware
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Prepare the text dataset for fine-tuning
train_path = 'bakery_for_finetuning.txt'
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=64  # Max token length for each training sample
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False  # Disable masked language modeling for GPT-2
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned-recipes-bakery',  # Where to save the model
    overwrite_output_dir=True,
    num_train_epochs=2,  # Number of training epochs
    per_device_train_batch_size=2,  # Small batch size for memory efficiency
    gradient_accumulation_steps=4,
    save_steps=1000,
    save_total_limit=1,
    logging_dir='./logs',  # Directory for logs
    logging_steps=500,
    evaluation_strategy="no",  # No validation during training
    fp16=False,  # Disable mixed precision for MPS
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('./gpt2-finetuned-recipes-bakery')
tokenizer.save_pretrained('./gpt2-finetuned-recipes-bakery')

print("Fine-tuning complete. Model saved to './gpt2-finetuned-recipes-bakery'")

/home/hrithik/myenv/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/home/hrithik/myenv/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
500,2.633400
1000,2.440600


Fine-tuning complete. Model saved to './gpt2-finetuned-recipes-bakery'
